In [2]:
import os
import glob
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten,GlobalAveragePooling2D


2023-08-18 23:46:25.975812: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-18 23:46:33.556126: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 23:46:53.075792: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


2023-08-18 23:52:40.467753: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-18 23:52:46.601446: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-18 23:52:46.601616: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
directory_path = "/home/e17358/4yp/MedNET/MedNET/createdModels/"


# Use the glob module to get a list of file paths in the specified directory
model_paths = glob.glob(os.path.join(directory_path, '*.h5'))
print(model_paths)

['/home/e17358/4yp/MedNET/MedNET/createdModels/densenet201_feature_extractor.h5', '/home/e17358/4yp/MedNET/MedNET/createdModels/densenet169_feature_extractor.h5', '/home/e17358/4yp/MedNET/MedNET/createdModels/mobilenet_feature_extractor.h5', '/home/e17358/4yp/MedNET/MedNET/createdModels/efficientnetb0_feature_extractor.h5', '/home/e17358/4yp/MedNET/MedNET/createdModels/densenet121_feature_extractor.h5', '/home/e17358/4yp/MedNET/MedNET/createdModels/vgg16_feature_extractor.h5', '/home/e17358/4yp/MedNET/MedNET/createdModels/inceptionv3_feature_extractor.h5', '/home/e17358/4yp/MedNET/MedNET/createdModels/inceptionresnetv2_feature_extractor.h5', '/home/e17358/4yp/MedNET/MedNET/createdModels/resnet50_feature_extractor.h5']


In [9]:
selected_model_path = '/home/e17358/4yp/MedNET/MedNET/createdModels/vgg16_feature_extractor.h5'

In [10]:
data_directory = "/home/e17358/4yp/MedNET/dataset/Breast Histopathology Images new/"

In [11]:

# Load the feature extractor model
feature_extractor = tf.keras.models.load_model(selected_model_path)
# feature_extractor.summary()


In [12]:
# Add GlobalAveragePooling2D layer to preserve shape information
x = GlobalAveragePooling2D()(feature_extractor.output)

# Add additional layers
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Create the final model
new_model = tf.keras.Model(inputs=feature_extractor.input, outputs=predictions)

# Compile the new model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the summary of the new model
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [13]:
# Data preprocessing and augmentation``
datagen = ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = datagen.flow_from_directory(
    '/home/e17358/4yp/MedNET/dataset/Breast Histopathology Images new/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 173491 images belonging to 2 classes.


In [14]:
datagen = ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_generator = datagen.flow_from_directory(
    '/home/e17358/4yp/MedNET/dataset/Breast Histopathology Images new/valid',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
)

Found 37178 images belonging to 2 classes.


In [ ]:
# Fine-tune the model
new_model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=valid_generator, validation_steps=len(valid_generator))

Epoch 1/10


2023-08-18 23:55:19.848774: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-08-18 23:56:01.295619: W tensorflow/tsl/framework/bfc_allocator.cc:366] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2023-08-18 23:56:01.527002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-18 23:56:02.500948: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x328b4e10 initialized for platform CUDA (this does not guarantee tha

1286/5422 [======>.......................] - ETA: 1:22:15 - loss: 0.4594 - accuracy: 0.7939

In [ ]:
new_model.save('medNet_VGG16.h5new_model

In [ ]:
new_model.summary()

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_generator = datagen.flow_from_directory(
    '/home/e17358/4yp/MedNET/dataset/Breast Histopathology Images new/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
results = new_model.evaluate(test_generator, steps=len(test_generator))

# Print the evaluation results (e.g., loss and accuracy)
print("Test loss:", results[0])
print("Test accuracy:", results[1])